# FeatureBook Alternative 1

In [1]:
import gc
import numpy as np
import pandas as pd
import xgboost as xgb
import pickle as pkl

import os 
from datetime import datetime
import pytz
from time import sleep

from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Input, Activation
from tensorflow.python.keras.layers import BatchNormalization,Add,Dropout
from tensorflow.python.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.losses import mean_absolute_error
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

/home/paperspace/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
os.listdir('.') # check and loop
datetime.now(pytz.timezone('Europe/Oslo')).strftime("%m.%d.%Y_%H.%M.%S")


'08.12.2019_13.36.44'

## Angle Features

In [3]:
def generate_angles(df, df_structures):
    # Map XYZ-coordinates to the dataframe
    def map_atom_info(df, df_merge, atom_idx):
        df = pd.merge(df, df_merge.drop_duplicates(subset=['molecule_name', 'atom_index']), how = 'left',
                      left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                      right_on = ['molecule_name',  'atom_index'])

        df = df.drop('atom_index', axis=1)

        return df

    for atom_idx in [0, 1]:
        df = map_atom_info(df, df_structures, atom_idx)
        df = df.rename(columns={
            'atom': f'atom_{atom_idx}',
            'x': f'x_{atom_idx}',
            'y': f'y_{atom_idx}',
            'z': f'z_{atom_idx}'})
        
        df_structures['c_x'] = df_structures.groupby('molecule_name')['x'].transform('mean')
        df_structures['c_y'] = df_structures.groupby('molecule_name')['y'].transform('mean')
        df_structures['c_z'] = df_structures.groupby('molecule_name')['z'].transform('mean')
        df_structures['atom_n'] = df_structures.groupby('molecule_name')['atom_index'].transform('max')
    
    # Calculate initial distances
    def calculate_init_distances(df):
        df['dx'] = df['x_1'] - df['x_0']
        df['dy'] = df['y_1'] - df['y_0']
        df['dz'] = df['z_1'] - df['z_0']
        df['distance'] = (df['dx']**2 + df['dy']**2 + df['dz']**2)**(1/2)
        return df

    df = calculate_init_distances(df)
    
    # Extend the distance calculations
    def extended_distance_calculations(df):
        df_temp = df.loc[:, ["molecule_name","atom_index_0","atom_index_1","distance","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()
        df_temp_ = df_temp.copy()
        df_temp_ = df_temp_.rename(columns={
            'atom_index_0': 'atom_index_1',
            'atom_index_1': 'atom_index_0',
            'x_0': 'x_1', 'y_0': 'y_1',
            'z_0': 'z_1', 'x_1': 'x_0',
            'y_1': 'y_0', 'z_1': 'z_0'})
        
        df_temp_all = pd.concat((df_temp, df_temp_), axis=0)
        
        df_temp_all["min_distance"] = df_temp_all.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('min')
        df_temp_all["max_distance"] = df_temp_all.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('max')
        
        df_temp = df_temp_all[df_temp_all["min_distance"] == df_temp_all["distance"]].copy()
        df_temp = df_temp.drop(['x_0', 'y_0', 'z_0', 'min_distance'], axis=1)
        df_temp = df_temp.rename(columns={
            'atom_index_0': 'atom_index',
            'atom_index_1': 'atom_index_closest',
            'distance': 'distance_closest',
            'x_1': 'x_closest',
            'y_1': 'y_closest',
            'z_1': 'z_closest'})
        
        for atom_idx in [0, 1]:
            df = map_atom_info(df, df_temp, atom_idx)
            df = df.rename(columns={
                'atom_index_closest': f'atom_index_closest_{atom_idx}',
                'distance_closest': f'distance_closest_{atom_idx}',
                'x_closest': f'x_closest_{atom_idx}',
                'y_closest': f'y_closest_{atom_idx}',
                'z_closest': f'z_closest_{atom_idx}'})
        
        df_temp = df_temp_all[df_temp_all["max_distance"] == df_temp_all["distance"]].copy()
        df_temp = df_temp.drop(['x_0', 'y_0', 'z_0', 'max_distance'], axis=1)
        df_temp = df_temp.rename(columns={
            'atom_index_0': 'atom_index',
            'atom_index_1': 'atom_index_farthest',
            'distance': 'distance_farthest',
            'x_1': 'x_farthest',
            'y_1': 'y_farthest',
            'z_1': 'z_farthest'})
        
        for atom_idx in [0, 1]:
            df = map_atom_info(df, df_temp, atom_idx)
            df = df.rename(columns={
                'atom_index_farthest': f'atom_index_farthest_{atom_idx}',
                'distance_farthest': f'distance_farthest_{atom_idx}',
                'x_farthest': f'x_farthest_{atom_idx}',
                'y_farthest': f'y_farthest_{atom_idx}',
                'z_farthest': f'z_farthest_{atom_idx}'})
        
        return df
    
    df = extended_distance_calculations(df)
    
    # Add angle features
    def add_angles(df):
        df["distance_center0"]=((df['x_0']-df['c_x'])**2+(df['y_0']-df['c_y'])**2+(df['z_0']-df['c_z'])**2)**(1/2)
        df["distance_center1"]=((df['x_1']-df['c_x'])**2+(df['y_1']-df['c_y'])**2+(df['z_1']-df['c_z'])**2)**(1/2)
        df["distance_c0"]=((df['x_0']-df['x_closest_0'])**2+(df['y_0']-df['y_closest_0'])**2+(df['z_0']-df['z_closest_0'])**2)**(1/2)
        df["distance_c1"]=((df['x_1']-df['x_closest_1'])**2+(df['y_1']-df['y_closest_1'])**2+(df['z_1']-df['z_closest_1'])**2)**(1/2)
        df["distance_f0"]=((df['x_0']-df['x_farthest_0'])**2+(df['y_0']-df['y_farthest_0'])**2+(df['z_0']-df['z_farthest_0'])**2)**(1/2)
        df["distance_f1"]=((df['x_1']-df['x_farthest_1'])**2+(df['y_1']-df['y_farthest_1'])**2+(df['z_1']-df['z_farthest_1'])**2)**(1/2)
        df["vec_center0_x"]=(df['x_0']-df['c_x'])/(df["distance_center0"]+1e-10)
        df["vec_center0_y"]=(df['y_0']-df['c_y'])/(df["distance_center0"]+1e-10)
        df["vec_center0_z"]=(df['z_0']-df['c_z'])/(df["distance_center0"]+1e-10)
        df["vec_center1_x"]=(df['x_1']-df['c_x'])/(df["distance_center1"]+1e-10)
        df["vec_center1_y"]=(df['y_1']-df['c_y'])/(df["distance_center1"]+1e-10)
        df["vec_center1_z"]=(df['z_1']-df['c_z'])/(df["distance_center1"]+1e-10)
        df["vec_c0_x"]=(df['x_0']-df['x_closest_0'])/(df["distance_c0"]+1e-10)
        df["vec_c0_y"]=(df['y_0']-df['y_closest_0'])/(df["distance_c0"]+1e-10)
        df["vec_c0_z"]=(df['z_0']-df['z_closest_0'])/(df["distance_c0"]+1e-10)
        df["vec_c1_x"]=(df['x_1']-df['x_closest_1'])/(df["distance_c1"]+1e-10)
        df["vec_c1_y"]=(df['y_1']-df['y_closest_1'])/(df["distance_c1"]+1e-10)
        df["vec_c1_z"]=(df['z_1']-df['z_closest_1'])/(df["distance_c1"]+1e-10)
        df["vec_f0_x"]=(df['x_0']-df['x_farthest_0'])/(df["distance_f0"]+1e-10)
        df["vec_f0_y"]=(df['y_0']-df['y_farthest_0'])/(df["distance_f0"]+1e-10)
        df["vec_f0_z"]=(df['z_0']-df['z_farthest_0'])/(df["distance_f0"]+1e-10)
        df["vec_f1_x"]=(df['x_1']-df['x_farthest_1'])/(df["distance_f1"]+1e-10)
        df["vec_f1_y"]=(df['y_1']-df['y_farthest_1'])/(df["distance_f1"]+1e-10)
        df["vec_f1_z"]=(df['z_1']-df['z_farthest_1'])/(df["distance_f1"]+1e-10)
        df["vec_x"]=(df['x_1']-df['x_0'])/df["distance"]
        df["vec_y"]=(df['y_1']-df['y_0'])/df["distance"]
        df["vec_z"]=(df['z_1']-df['z_0'])/df["distance"]
        df["cos_c0_c1"]=df["vec_c0_x"]*df["vec_c1_x"]+df["vec_c0_y"]*df["vec_c1_y"]+df["vec_c0_z"]*df["vec_c1_z"]
        df["cos_f0_f1"]=df["vec_f0_x"]*df["vec_f1_x"]+df["vec_f0_y"]*df["vec_f1_y"]+df["vec_f0_z"]*df["vec_f1_z"]
        df["cos_center0_center1"]=df["vec_center0_x"]*df["vec_center1_x"]+df["vec_center0_y"]*df["vec_center1_y"]+df["vec_center0_z"]*df["vec_center1_z"]
        df["cos_c0"]=df["vec_c0_x"]*df["vec_x"]+df["vec_c0_y"]*df["vec_y"]+df["vec_c0_z"]*df["vec_z"]
        df["cos_c1"]=df["vec_c1_x"]*df["vec_x"]+df["vec_c1_y"]*df["vec_y"]+df["vec_c1_z"]*df["vec_z"]
        df["cos_f0"]=df["vec_f0_x"]*df["vec_x"]+df["vec_f0_y"]*df["vec_y"]+df["vec_f0_z"]*df["vec_z"]
        df["cos_f1"]=df["vec_f1_x"]*df["vec_x"]+df["vec_f1_y"]*df["vec_y"]+df["vec_f1_z"]*df["vec_z"]
        df["cos_center0"]=df["vec_center0_x"]*df["vec_x"]+df["vec_center0_y"]*df["vec_y"]+df["vec_center0_z"]*df["vec_z"]
        df["cos_center1"]=df["vec_center1_x"]*df["vec_x"]+df["vec_center1_y"]*df["vec_y"]+df["vec_center1_z"]*df["vec_z"]
        # original
        df=df.drop(['vec_c0_x','vec_c0_y','vec_c0_z','vec_c1_x','vec_c1_y','vec_c1_z',
                    'vec_f0_x','vec_f0_y','vec_f0_z','vec_f1_x','vec_f1_y','vec_f1_z',
                    'vec_center0_x','vec_center0_y','vec_center0_z','vec_center1_x','vec_center1_y','vec_center1_z',
                    'vec_x','vec_y','vec_z'], axis=1)
        # extra 
        df=df.drop(['dy', 'dx', 'dz',
                    'x_0', 'x_1', 'c_x', 'x_closest_0', 'x_closest_1', 'x_farthest_0', 'x_farthest_1',
                    'y_0', 'y_1', 'c_y', 'y_closest_0', 'y_closest_1', 'y_farthest_0', 'y_farthest_1',
                    'z_0', 'z_1', 'c_z', 'z_closest_0', 'z_closest_1', 'z_farthest_0', 'z_farthest_1'], axis=1)
        return df
    
    df = add_angles(df)
    
    return df

## General Methods

In [4]:
coupling_types = ["1JHC", "1JHN", "2JHH", "2JHC", "2JHN", "3JHH", "3JHC", "3JHN"]

def select_wanted(df, wanted_rows):
    df = df[df["type"] == wanted_rows]
    return df

def memory_optimization(dfs):
    for df in dfs:
        del df
    gc.collect()

def merge_multiple(df, df_merge, atom_idx):
    df = pd.merge(df, df_merge, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    return df

def merge_single(df, df_merge):
    df = pd.merge(df, df_merge, how = 'left',
                  left_on  = ['molecule_name'],
                  right_on = ['molecule_name'])
        
    return df

def merge_custom(df, df_merge):
    df = pd.merge(df, df_merge, how = 'left',
                 left_on  = ['id', 'molecule_name'],
                 right_on = ['id', 'molecule_name'])
    
    return df

def drop_col(df, cols):
    columns = list(df.columns.values)
    
    for col in cols:
        if col in columns:
            df = df.drop(col, axis=1)
            
    return df

## Append Custom Features to DataFrame

In [5]:
def loader(df, train_or_test):
    # Sorted distances
    df_sorted_distances = pd.read_csv('./input/generated/sorted_distances.csv')
    df = merge_multiple(df, df_sorted_distances, 0)
    df = merge_multiple(df, df_sorted_distances, 1)
    memory_optimization([df_sorted_distances])
    
    # Dipole moments
    df_dipole_moments = pd.read_csv(f'./input/generated/{train_or_test}_ob_dipoles_mmff44.csv')
    df_dipole_moments = df_dipole_moments.loc[:, ~df_dipole_moments.columns.str.contains('^Unnamed')]
    df_dipole_moments['size'] = (df_dipole_moments['X']**2 + df_dipole_moments['Y']**2 + df_dipole_moments['Z']**2)**(1/2)
    df = merge_single(df, df_dipole_moments)
    memory_optimization([df_dipole_moments])

    # Mulliken charges
    df_mulliken_charges = pd.read_csv(f'./input/generated/best_ob_mulliken_{train_or_test}.csv')
    df_mulliken_charges = df_mulliken_charges.loc[:, ~df_mulliken_charges.columns.str.contains('^Unnamed')]
    df = merge_multiple(df, df_mulliken_charges, 0)
    df = merge_multiple(df, df_mulliken_charges, 1)
    memory_optimization([df_mulliken_charges])
    
    # Number of atoms and max distances
    df_natoms_maxdist = pd.read_csv(f'./input/generated/natoms_maxdist.csv')
    df = merge_multiple(df, df_natoms_maxdist, 0)
    df = df.drop(['atom'], axis = 1)
    memory_optimization([df_natoms_maxdist])

    # Coupling distances
    df_coupling_distances = pd.read_csv(f'./input/generated/coupling_distances.csv')
    df = merge_custom(df, df_coupling_distances)
    memory_optimization([df_coupling_distances])
    
    df = drop_col(df, ['atom_index_0', 'atom_index_1', 'atom_index_closest_0', 'atom_index_closest_1', 
                       'atom_index_farthest_0', 'atom_index_farthest_1','X', 'Y', 'Z']) #'id',
    
    return df

## Convert Data Types

In [6]:
def convert_df(df):
    atom_representation = {
        'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9,
    }

    coupling_representation = {
        '1JHC': 1, '1JHN': 2, '2JHH': 3, '2JHC': 4, '2JHN': 5, '3JHH': 6, '3JHC': 7, '3JHN': 8,
    }

    df = df.replace({
        'type': coupling_representation,
        'atom_x': atom_representation,
        'atom_y': atom_representation,
        'atom_0': atom_representation,
        'atom_1': atom_representation,
    })
    df.type = df.type.astype('int64')
    df.atom_x = df.atom_x.astype('int64')
    df.atom_y = df.atom_y.astype('int64')
    df.atom_0 = df.atom_0.astype('int64')
    df.atom_1 = df.atom_1.astype('int64')
    
    df['molecule_name'] = df['molecule_name'].astype('category')
    df_cats = df.select_dtypes(['category']).columns
    df[df_cats] = df[df_cats].apply(lambda x: x.cat.codes)
    
    return df

## Column Drop Test

In [7]:
def col_dropping(df):
    df = df.drop(['molecule_name'], axis=1)
    df = df.drop(['atom_n'], axis=1)
    
    # Drop distances
    df = drop_col(df, ['d10_x', 'd11_x', 'd12_x', 'd13_x', 'd14_x', 'd15_x', 'd16_x', 'd17_x', 'd18_x', 'd19_x', 'd20_x', 'd21_x', 'd22_x', 'd23_x', 'd24_x', 'd25_x', 'd26_x', 'd27_x', 'd28_x',
                  'd10_y', 'd11_y', 'd12_y', 'd13_y', 'd14_y', 'd15_y', 'd16_y', 'd17_y', 'd18_y', 'd19_y', 'd20_y', 'd21_y', 'd22_y', 'd23_y', 'd24_y', 'd25_y', 'd26_y', 'd27_y', 'd28_y'])
    
    # Drop types
    df = drop_col(df, ['t10_x', 't11_x', 't12_x', 't13_x', 't14_x', 't15_x', 't16_x', 't17_x', 't18_x', 't19_x', 't20_x', 't21_x', 't22_x', 't23_x', 't24_x', 't25_x', 't26_x', 't27_x', 't28_x',
                  't10_y', 't11_y', 't12_y', 't13_y', 't14_y', 't15_y', 't16_y', 't17_y', 't18_y', 't19_y', 't20_y', 't21_y', 't22_y', 't23_y', 't24_y', 't25_y', 't26_y', 't27_y', 't28_y'])
    
    df = drop_col(df, ['atom_0', 'atom_1', 'eem_x', 'eem_y', 'X', 'Y', 'Z'])
    
    return df

## Features Master Organizer

In [8]:
def feature_master(coupling_type):
    # Load structures
    df_structures = pd.read_csv('./input/structures.csv')
    
    # Generate train for current coupling type
    df_train = pd.read_csv('./input/train.csv')
    df_train = select_wanted(df_train, coupling_type)
    df_train = generate_angles(df_train, df_structures)
    df_train = loader(df_train, 'train')
    df_train = convert_df(df_train)
    df_train = col_dropping(df_train)
    #print("df_train: \n", df_train.head(1))
    
    df_structures = None
    df_structures = pd.read_csv('./input/structures.csv')
    
    # Generate test for current coupling type
    df_test = pd.read_csv('./input/test.csv')
    df_test = select_wanted(df_test, coupling_type)
    df_test = generate_angles(df_test, df_structures)
    df_test = loader(df_test, 'test')
    df_test = convert_df(df_test)
    df_test = col_dropping(df_test)
    #print("df_test: \n", df_test.head(1))
    
    # Return results
    return df_train, df_test

## Split Training and Validation

In [9]:
def split(df_train):
    train_X, validation_X = train_test_split(df_train, test_size = 0.1, random_state = 0)

    train_X = train_X.reset_index()
    validation_X = validation_X.reset_index()

    train_y = train_X['scalar_coupling_constant']
    train_y = train_y.replace([np.inf, -np.inf], np.nan)
    train_y = train_y.reset_index()
    train_y = train_y.drop(['index'], axis = 1)
    validation_y = validation_X['scalar_coupling_constant']
    validation_y = validation_y.replace([np.inf, -np.inf], np.nan)
    validation_y = validation_y.reset_index()
    validation_y = validation_y.drop(['index'], axis = 1)

    train_X = train_X.drop('scalar_coupling_constant', axis = 1)
    validation_X = validation_X.drop('scalar_coupling_constant', axis = 1)
    
    train_X = train_X.drop(['index'], axis = 1)
    validation_X = validation_X.drop(['index'], axis = 1)

    #print(train_X.head())
    #print(train_y.head())
    #print(validation_X.head())
    #print(validation_y.head())
    
    return train_X, train_y, validation_X, validation_y

## NN Training

In [36]:
def create_nn_model(input_shape):
    # input layer
    inp = Input(shape=(input_shape,))

    # first hidden layer
    x = Dense(256, kernel_initializer = 'he_normal')(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    # second hidden layer
    x = Dense(512, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    # third hidden layer
    x = Dense(1024, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    # fourth hidden layer
    x = Dense(512, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    # fifth hidden layer
    x = Dense(256, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    # sixth hidden layer
    x = Dense(128, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    # seventh hidden layer
    x = Dense(128, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    # eight hidden layer
    x = Dense(64, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    
    # output layer scalar_coupling_constant
    out = Dense(1, activation="linear")(x)   
    model = Model(inputs=inp, outputs=out)
    return model


In [37]:
def nn_train(coupling_type, train_X, train_y, validation_X, validation_y):
    epoch_n = 2000
    verbose = 1
    batch_size = 2048
    model_name = './models/weekend/nn/coupling_model_%s_NN.hdf5' % coupling_type
    
    nn_model = create_nn_model(train_X.shape[1])

    nn_model.compile(loss = 'mae', optimizer = Adam())
    
    es = callbacks.EarlyStopping(monitor = 'loss', min_delta = 0.00001, patience = 64,
                                 verbose = verbose, mode = 'auto', restore_best_weights = True)
    
    rlr = callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 32,
                                      min_lr = 1e-7, mode = 'auto', verbose = verbose)
    
    sv_mod = callbacks.ModelCheckpoint(model_name, monitor = 'val_loss', save_best_only = True,
                                       save_weights_only = True, restore_best_weights = True)
    
    history = nn_model.fit(x = train_X.values, y = train_y['scalar_coupling_constant'].values, 
                           validation_data = (validation_X.values, validation_y['scalar_coupling_constant'].values),
                           callbacks = [es, rlr, sv_mod], epochs = epoch_n, batch_size = batch_size, verbose = verbose)
    
    nn_model.save_weights(model_name)

    return nn_model

## XGB Training

In [38]:
def xgb_train(coupling_type, train_X, train_y, validation_X, validation_y):

    model_name_wrt = ('./models/weekend/xgb/coupling_model_%s_XGB.hdf5' % coupling_type)

    xgb_model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=10000, n_jobs=-1,
       nthread=None, objective='reg:squarederror', random_state=101, reg_alpha=2,
       reg_lambda=0.2, scale_pos_weight=1, seed=None, silent=False, 
       subsample=1)

    xgb_model.fit(train_X, train_y, eval_set = [(validation_X, validation_y)], eval_metric = 'mae', 
              early_stopping_rounds = 32, verbose = True)   
    xgb_model.save_model(model_name_wrt)
    #joblib.dump(xgb_model, model_name_wrt) 
    
    return xgb_model

In [39]:
def importance(xgb_model, train_X):
    input_features = train_X.columns.values

    import operator
    feat_imp = xgb_model.feature_importances_
    np.split(feat_imp, len(input_features))
    feat_imp_dict = {}
    for i in range(0, len(input_features)):
        feat_imp_dict[feat_imp[i]]=input_features[i]

    sorted_feats = sorted(feat_imp_dict.items(), key=operator.itemgetter(0))

    for i in range( len(sorted_feats)-1, 0, -1):
        print(sorted_feats[i])

## Training, Blending and Submission

In [40]:
def val_blending_list( nn_val_predict, xgb_val_predict, validation_y ):

    nn_val_pred_error = (validation_y - nn_val_predict)
    xgb_val_pred_error = (validation_y - xgb_val_predict )

    val_error_corr = np.corrcoef(nn_val_pred_error, xgb_val_pred_error)

    print(f'error correlation: {val_error_corr[0][1]}, error difference {xgb_accuracy - nn_accuracy}')
    log_accuracy = 0
    log_accuracy_list = []
    val_predict = np.array([])

    for alpha in np.arange(0,1.1,0.1):
        nn_val_predict_scaled = alpha*nn_val_predict
        xgb_val_predict_scaled = (1-alpha)*xgb_val_predict
        val_predict = nn_val_predict_scaled + xgb_val_predict_scaled
        log_accuracy=np.log(np.mean(np.abs(validation_y-val_predict)))
        log_accuracy_list.append(log_accuracy)

    return log_accuracy_list

In [ ]:
load = False

df_test = pd.read_csv('submissions/submission_best.csv')
test_prediction=df_test['scalar_coupling_constant']
df_test_full = pd.read_csv('./input/test.csv')

start_time=datetime.now()

val_score={"1JHC": np.inf, "1JHN": np.inf, "2JHH": np.inf, "2JHC": np.inf, 
          "2JHN": np.inf, "3JHH": np.inf, "3JHC": np.inf, "3JHN": np.inf}

coupling_types = ["1JHN", "1JHC", "2JHH", "2JHC", "2JHN", "3JHH", "3JHC", "3JHN"]

for coupling_type in coupling_types:
    time = datetime.now(pytz.timezone('Europe/Oslo')).strftime("%m.%d.%Y_%H.%M.%S")
    print(f'Predicting {coupling_type} out of {coupling_types} at {time}')
    
    df_train, df_test = feature_master(coupling_type)
    # Split into train and validation
    train_X, train_y, validation_X, validation_y = split(df_train)
    
    ################################# NN ########################################
    
    if load:         
        model_name_rd_nn = ('models/nn/nn_model_%s.hdf5' % coupling_type)
        nn_model=create_nn_model(validation_X.shape[1]) # vals
        print(f'loading weights from {model_name_rd_nn}')
        nn_model.load_weights(model_name_rd_nn)
    else:
        # Train and return trained model
        nn_model = nn_train(coupling_type, train_X, train_y, validation_X, validation_y)
    
    nn_val_predict = nn_model.predict(validation_X)

    nn_accuracy = np.log(np.mean(np.abs(validation_y.values-nn_val_predict)))
    print(f"validation score for {coupling_type} is {nn_accuracy} with NN\n")
    
    ################################# XGB ########################################

    if load:
        model_name_rd_xgb = ('models/xgb_jl/featurebook_%s.joblib.dat' % coupling_type)

        xgb_model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
           colsample_bytree=1, gamma=0, importance_type='gain',
           learning_rate=0.1, max_delta_step=0, max_depth=9,
           min_child_weight=1, missing=None, n_estimators=10000, n_jobs=-1,
           nthread=None, objective='reg:squarederror', random_state=42, reg_alpha=2,
           reg_lambda=0.2, scale_pos_weight=1, seed=None, silent=False,
           subsample=1)

        print(f'loading weights from {model_name_rd_xgb}')
        #xgb_model.load_model(model_name_rd_xgb)
        xgb_model= joblib.load(model_name_rd_xgb)
    else:
        xgb_model = xgb_train(coupling_type, train_X, train_y, validation_X, validation_y)   
        
    xgb_val_predict = xgb_model.predict(validation_X)
    
    memory_optimization([train_X, train_y, validation_X])
    gc.collect()
    
    validation_y = validation_y['scalar_coupling_constant'].values

    diff = validation_y-xgb_val_predict
    xgb_accuracy=np.log(np.mean(np.abs(diff)))

    print(f"validation score for {coupling_type} is {xgb_accuracy} with XGB\n")
     
    ############################## BLEND ##################################
        
    nn_val_predict = np.transpose(nn_val_predict[:,0])
    log_accuracy_list = val_blending_list(nn_val_predict, xgb_val_predict, validation_y)
    
    print(f'accuracy_list with alphas for {coupling_type}:\n {log_accuracy_list}')
    alpha_i=np.argmin(log_accuracy_list)
    log_accuracy = log_accuracy_list[alpha_i]
    val_score[coupling_type] =(log_accuracy)
    alpha = alpha_i*0.1
    
    print(f'blending with alpha = {alpha}, final accuracy  for {coupling_type} = {log_accuracy}') 
    
    ############################## PREDICT ##################################
    print('predicting NN')
    nn_test_predict = nn_model.predict(df_test)
    nn_test_predict_scaled=alpha * nn_test_predict
    nn_test_predict_scaled = np.transpose(nn_test_predict_scaled[:,0])
    
    print('predicting XGB')
    xgb_test_predict = xgb_model.predict(df_test)
    xgb_test_predict_scaled = (1-alpha) * xgb_test_predict
    
    test_predict = nn_test_predict_scaled + xgb_test_predict_scaled
    test_prediction[df_test_full["type"]==coupling_type]=test_predict

    memory_optimization([df_test, nn_model, nn_val_predict, xgb_model, xgb_val_predict, nn_test_predict, 
                         nn_test_predict_scaled, xgb_test_predict, xgb_test_predict_scaled, test_predict])
    gc.collect()
    
val_score_total=sum(val_score.values())/len(val_score.keys())
print(f'total cv score is {val_score_total}\n')

Predicting 1JHN out of ['1JHN', '1JHC', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN'] at 08.12.2019_14.15.32
Train on 39026 samples, validate on 4337 samples
Epoch 1/2000
39026/39026 [==============================] - 4s 103us/sample - loss: 46.2046 - val_loss: 28.2206
Epoch 2/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 45.2054 - val_loss: 34.7711
Epoch 3/2000
39026/39026 [==============================] - 1s 14us/sample - loss: 44.0665 - val_loss: 27.4963
Epoch 4/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 43.0055 - val_loss: 46.3534
Epoch 5/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 42.0428 - val_loss: 35.2963
Epoch 6/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 41.0798 - val_loss: 28.6252
Epoch 7/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 40.0787 - val_loss: 31.3187
Epoch 8/2000
39026/39026 [==============================] - 0s

39026/39026 [==============================] - 0s 12us/sample - loss: 9.2169 - val_loss: 9.6671
Epoch 74/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.3093 - val_loss: 9.5969
Epoch 75/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.2308 - val_loss: 9.8563
Epoch 76/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.2651 - val_loss: 10.0436
Epoch 77/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.2284 - val_loss: 9.7169
Epoch 78/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.3145 - val_loss: 9.6750
Epoch 79/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.2840 - val_loss: 9.7501
Epoch 80/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.2556 - val_loss: 9.6508
Epoch 81/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.2778 - val_loss: 9.8337
Epoch 82/2000
39026/390

39026/39026 [==============================] - 0s 12us/sample - loss: 9.1069 - val_loss: 8.9046
Epoch 148/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.1407 - val_loss: 8.9998
Epoch 149/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0753 - val_loss: 9.1121
Epoch 150/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0952 - val_loss: 9.1144
Epoch 151/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.1424 - val_loss: 9.0026
Epoch 152/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0841 - val_loss: 8.8108
Epoch 153/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0577 - val_loss: 9.1454
Epoch 154/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0878 - val_loss: 9.4318
Epoch 155/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.1003 - val_loss: 9.7005
Epoch 156/2000
3

39026/39026 [==============================] - 1s 13us/sample - loss: 9.1051 - val_loss: 8.1175
Epoch 219/2000
39026/39026 [==============================] - 1s 13us/sample - loss: 9.0588 - val_loss: 8.1130
Epoch 220/2000
39026/39026 [==============================] - 1s 13us/sample - loss: 9.0561 - val_loss: 8.1119
Epoch 221/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.0249 - val_loss: 8.1139
Epoch 222/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.0297 - val_loss: 8.1129
Epoch 223/2000
39026/39026 [==============================] - 1s 14us/sample - loss: 9.0677 - val_loss: 8.1111
Epoch 224/2000
39026/39026 [==============================] - 1s 14us/sample - loss: 9.0079 - val_loss: 8.1055
Epoch 225/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.0772 - val_loss: 8.1067
Epoch 226/2000
39026/39026 [==============================] - 1s 13us/sample - loss: 9.0553 - val_loss: 8.1047
Epoch 227/2000
3

39026/39026 [==============================] - 0s 11us/sample - loss: 9.0538 - val_loss: 8.1109
Epoch 291/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.0532 - val_loss: 8.1120
Epoch 292/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0524 - val_loss: 8.1043
Epoch 293/2000
39026/39026 [==============================] - 0s 11us/sample - loss: 9.0323 - val_loss: 8.0941
Epoch 294/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0324 - val_loss: 8.0917
Epoch 295/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0132 - val_loss: 8.0898
Epoch 296/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0063 - val_loss: 8.0877
Epoch 297/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0429 - val_loss: 8.0878
Epoch 298/2000
39026/39026 [==============================] - 0s 12us/sample - loss: 9.0348 - val_loss: 8.0857
Epoch 299/2000
3

...

[7889]	validation_0-mae:0.301217
[7890]	validation_0-mae:0.301214
[7891]	validation_0-mae:0.301213
[7892]	validation_0-mae:0.301208
[7893]	validation_0-mae:0.301205
[7894]	validation_0-mae:0.301203
[7895]	validation_0-mae:0.3012
[7896]	validation_0-mae:0.301195
[7897]	validation_0-mae:0.301194
[7898]	validation_0-mae:0.301195
[7899]	validation_0-mae:0.301193
[7900]	validation_0-mae:0.30119
[7901]	validation_0-mae:0.301186
[7902]	validation_0-mae:0.301183
[7903]	validation_0-mae:0.30118
[7904]	validation_0-mae:0.301177
[7905]	validation_0-mae:0.301175
[7906]	validation_0-mae:0.301174
[7907]	validation_0-mae:0.30117
[7908]	validation_0-mae:0.301168
[7909]	validation_0-mae:0.301167
[7910]	validation_0-mae:0.301166
[7911]	validation_0-mae:0.301162
[7912]	validation_0-mae:0.301158
[7913]	validation_0-mae:0.301158
[7914]	validation_0-mae:0.301156
[7915]	validation_0-mae:0.301153
[7916]	validation_0-mae:0.301151
[7917]	validation_0-mae:0.30115
[7918]	validation_0-mae:0.30115
[7919]	validation

[8139]	validation_0-mae:0.30071
[8140]	validation_0-mae:0.300709
[8141]	validation_0-mae:0.300707
[8142]	validation_0-mae:0.300703
[8143]	validation_0-mae:0.300701
[8144]	validation_0-mae:0.300701
[8145]	validation_0-mae:0.300701
[8146]	validation_0-mae:0.300699
[8147]	validation_0-mae:0.300697
[8148]	validation_0-mae:0.300696
[8149]	validation_0-mae:0.300696
[8150]	validation_0-mae:0.300695
[8151]	validation_0-mae:0.300692
[8152]	validation_0-mae:0.300692
[8153]	validation_0-mae:0.300688
[8154]	validation_0-mae:0.300687
[8155]	validation_0-mae:0.300684
[8156]	validation_0-mae:0.300682
[8157]	validation_0-mae:0.300679
[8158]	validation_0-mae:0.300674
[8159]	validation_0-mae:0.30067
[8160]	validation_0-mae:0.300666
[8161]	validation_0-mae:0.300663
[8162]	validation_0-mae:0.30066
[8163]	validation_0-mae:0.300657
[8164]	validation_0-mae:0.300657
[8165]	validation_0-mae:0.300652
[8166]	validation_0-mae:0.300651
[8167]	validation_0-mae:0.300651
[8168]	validation_0-mae:0.30065
[8169]	validat

[8389]	validation_0-mae:0.300153
[8390]	validation_0-mae:0.300149
[8391]	validation_0-mae:0.300149
[8392]	validation_0-mae:0.300147
[8393]	validation_0-mae:0.300147
[8394]	validation_0-mae:0.300144
[8395]	validation_0-mae:0.300143
[8396]	validation_0-mae:0.300142
[8397]	validation_0-mae:0.30014
[8398]	validation_0-mae:0.300139
[8399]	validation_0-mae:0.300137
[8400]	validation_0-mae:0.300136
[8401]	validation_0-mae:0.300136
[8402]	validation_0-mae:0.300133
[8403]	validation_0-mae:0.300131
[8404]	validation_0-mae:0.30013
[8405]	validation_0-mae:0.300125
[8406]	validation_0-mae:0.300122
[8407]	validation_0-mae:0.300119
[8408]	validation_0-mae:0.300119
[8409]	validation_0-mae:0.300116
[8410]	validation_0-mae:0.300113
[8411]	validation_0-mae:0.300112
[8412]	validation_0-mae:0.300111
[8413]	validation_0-mae:0.30011
[8414]	validation_0-mae:0.300109
[8415]	validation_0-mae:0.300107
[8416]	validation_0-mae:0.300105
[8417]	validation_0-mae:0.300103
[8418]	validation_0-mae:0.300102
[8419]	valida

[8639]	validation_0-mae:0.299633
[8640]	validation_0-mae:0.299632
[8641]	validation_0-mae:0.299631
[8642]	validation_0-mae:0.29963
[8643]	validation_0-mae:0.299629
[8644]	validation_0-mae:0.299627
[8645]	validation_0-mae:0.299627
[8646]	validation_0-mae:0.299626
[8647]	validation_0-mae:0.299625
[8648]	validation_0-mae:0.299623
[8649]	validation_0-mae:0.299621
[8650]	validation_0-mae:0.29962
[8651]	validation_0-mae:0.299619
[8652]	validation_0-mae:0.299618
[8653]	validation_0-mae:0.299619
[8654]	validation_0-mae:0.299616
[8655]	validation_0-mae:0.299614
[8656]	validation_0-mae:0.299614
[8657]	validation_0-mae:0.299613
[8658]	validation_0-mae:0.299613
[8659]	validation_0-mae:0.299611
[8660]	validation_0-mae:0.29961
[8661]	validation_0-mae:0.299609
[8662]	validation_0-mae:0.29961
[8663]	validation_0-mae:0.299609
[8664]	validation_0-mae:0.299606
[8665]	validation_0-mae:0.299606
[8666]	validation_0-mae:0.2996
[8667]	validation_0-mae:0.2996
[8668]	validation_0-mae:0.299599
[8669]	validation_

[8889]	validation_0-mae:0.299229
[8890]	validation_0-mae:0.299228
[8891]	validation_0-mae:0.299229
[8892]	validation_0-mae:0.299226
[8893]	validation_0-mae:0.299225
[8894]	validation_0-mae:0.299224
[8895]	validation_0-mae:0.299222
[8896]	validation_0-mae:0.299221
[8897]	validation_0-mae:0.299219
[8898]	validation_0-mae:0.299219
[8899]	validation_0-mae:0.299217
[8900]	validation_0-mae:0.299216
[8901]	validation_0-mae:0.299214
[8902]	validation_0-mae:0.299209
[8903]	validation_0-mae:0.299206
[8904]	validation_0-mae:0.299203
[8905]	validation_0-mae:0.299202
[8906]	validation_0-mae:0.299202
[8907]	validation_0-mae:0.299196
[8908]	validation_0-mae:0.299194
[8909]	validation_0-mae:0.29919
[8910]	validation_0-mae:0.299189
[8911]	validation_0-mae:0.299189
[8912]	validation_0-mae:0.299186
[8913]	validation_0-mae:0.299185
[8914]	validation_0-mae:0.299181
[8915]	validation_0-mae:0.299178
[8916]	validation_0-mae:0.299178
[8917]	validation_0-mae:0.299177
[8918]	validation_0-mae:0.299176
[8919]	vali

## Make submission csv

In [ ]:
def submit(predictions):
    submit = pd.read_csv('submissions/submission_best.csv')  
    submit["scalar_coupling_constant"] = predictions
    submit.to_csv("submissions/submission_blended_full2.csv", index=False)

In [ ]:
submit(test_prediction)

time = datetime.now(pytz.timezone('Europe/Oslo')).strftime("%m.%d.%Y_%H.%M.%S")
print(f'Notebook EoF reached at {time} and submission saved')

## Testing Area